# [602. Friend Requests II: Who Has the Most Friends](https://leetcode.com/problems/friend-requests-ii-who-has-the-most-friends/?envType=study-plan-v2&envId=top-sql-50)

Table: RequestAccepted

<pre>+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| requester_id   | int     |
| accepter_id    | int     |
| accept_date    | date    |
+----------------+---------+</pre>
(requester_id, accepter_id) is the primary key (combination of columns with unique values) for this table.
This table contains the ID of the user who sent the request, the ID of the user who received the request, and the date when the request was accepted.
 

Write a solution to find the people who have the most friends and the most friends number.

The test cases are generated so that only one person has the most friends.

The result format is in the following example.

 

Example 1:

Input: 
RequestAccepted table:
<pre>+--------------+-------------+-------------+
| requester_id | accepter_id | accept_date |
+--------------+-------------+-------------+
| 1            | 2           | 2016/06/03  |
| 1            | 3           | 2016/06/08  |
| 2            | 3           | 2016/06/08  |
| 3            | 4           | 2016/06/09  |
+--------------+-------------+-------------+</pre>
Output: 
<pre>+----+-----+
| id | num |
+----+-----+
| 3  | 3   |
+----+-----+</pre>
Explanation: 
The person with id 3 is a friend of people 1, 2, and 4, so he has three friends in total, which is the most number than any others.
 

Follow up: In the real world, multiple people could have the same most number of friends. Could you find all these people in this case?

In [1]:
# pandas schema

import pandas as pd

data = [[1, 2, '2016/06/03'], [1, 3, '2016/06/08'], [2, 3, '2016/06/08'], [3, 4, '2016/06/09']]
request_accepted = pd.DataFrame(data, columns=['requester_id', 'accepter_id', 'accept_date']).astype(
    {'requester_id': 'Int64', 'accepter_id': 'Int64', 'accept_date': 'datetime64[ns]'})

In [2]:
#to spark dataframe
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

request_accepted_df = spark.createDataFrame(request_accepted)
request_accepted_df.show()

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/06 17:04:43 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/06 17:04:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/06 17:04:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+------------+-----------+-------------------+
|requester_id|accepter_id|        accept_date|
+------------+-----------+-------------------+
|           1|          2|2016-06-03 00:00:00|
|           1|          3|2016-06-08 00:00:00|
|           2|          3|2016-06-08 00:00:00|
|           3|          4|2016-06-09 00:00:00|
+------------+-----------+-------------------+


In [3]:
# solving in spark df
import pyspark.sql.functions as F

requester_count_df = request_accepted_df\
    .groupBy(F.col('requester_id').alias('id'))\
    .agg(F.count('requester_id').alias('friends'))
requester_count_df.show()

+---+-------+
| id|friends|
+---+-------+
|  1|      2|
|  2|      1|
|  3|      1|
+---+-------+


In [4]:
accepter_count_df = request_accepted_df\
    .groupBy(F.col('accepter_id').alias('id'))\
    .agg(F.count('accepter_id').alias('friends'))
accepter_count_df.show()

+---+-------+
| id|friends|
+---+-------+
|  2|      1|
|  3|      2|
|  4|      1|
+---+-------+


In [5]:
from pyspark.sql import Window

id_friends_df = requester_count_df\
    .unionAll(accepter_count_df)\
    .groupBy('id')\
    .agg(F.sum(F.col('friends')).alias('friends'))
id_friends_df.show()

+---+-------+
| id|friends|
+---+-------+
|  1|      2|
|  3|      3|
|  2|      2|
|  4|      1|
+---+-------+


In [6]:
max_friend_df = id_friends_df.select('friends').agg(F.max('friends'))
max_friends_value = max_friend_df.collect()[0][0]

# this solution will work for the 'follow up' scenario mentioned on description
final_df = id_friends_df.filter(F.col('friends')==max_friends_value)\
    .select('id',F.col('friends').alias('num'))
final_df.show()

+---+---+
| id|num|
+---+---+
|  3|  3|
+---+---+


In [7]:
# Solving in Spark SQL

request_accepted_df.createOrReplaceTempView('RequestAccepted')

spark.sql('''
with id_friends as (
select id, sum(friends) as friends from
(   
select requester_id as id, count(requester_id) as friends 
from requestaccepted group by requester_id
    union all
select accepter_id as id, count(accepter_id) as friends
from requestaccepted group by accepter_id
) tbl
group by id
)
select id, friends as num
from id_friends 
where (select max(friends) from id_friends) = friends;
''').show()

+---+---+
| id|num|
+---+---+
|  3|  3|
+---+---+


In [8]:
spark.stop()